In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
db_url = os.getenv('DATABASE')
db_url

In [42]:
engine = create_engine(db_url)

In [43]:
raw_df = pd.read_excel("online_retail_data.xlsx") #Uncleaned data

In [47]:
raw_df.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom


In [44]:
raw_df['InvoiceDate'] = pd.to_datetime(raw_df['InvoiceDate']).dt.normalize()

In [45]:
raw_df.to_sql('Raw_Retail_data', engine, if_exists='append', index=False, schema='online_retails', method= 'multi', chunksize=5000)

541909

In [13]:
df = pd.read_excel("online_retail_data.xlsx") #Cleaned data goes into df

In [14]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.normalize()

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392731 entries, 0 to 541908
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392731 non-null  object        
 1   StockCode    392731 non-null  object        
 2   Description  392731 non-null  object        
 3   Quantity     392731 non-null  int64         
 4   InvoiceDate  392731 non-null  datetime64[ns]
 5   UnitPrice    392731 non-null  float64       
 6   CustomerID   392731 non-null  object        
 7   Country      392731 non-null  object        
 8   Revenue      392731 non-null  float64       
 9   Year         392731 non-null  int32         
 10  Month        392731 non-null  int32         
 11  Day          392731 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(3), int64(1), object(5)
memory usage: 34.5+ MB


In [15]:
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom


In [16]:
df['CustomerID'] = df['CustomerID'].astype('object')  

In [17]:
df.shape

(541909, 8)

In [19]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Quantity and Price Validation

In [ ]:
#print((df['Quantity'] < 1).sum())

0


In [ ]:
#print((df['UnitPrice'] < 0).sum())

0


In [22]:
df = df[df['Quantity'] >= 1]

In [24]:
df = df[df['UnitPrice'] >= 0]

Additional Cleaning

In [26]:
df['InvoiceNo'].astype(str).str[0].value_counts()

InvoiceNo
5    531282
A         1
Name: count, dtype: int64

In [ ]:
#df[df['InvoiceNo'].astype(str).str.startswith('C')] #There are none because they were removed during the Qty and Price cleaning

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [29]:
df['CustomerID'].isna().sum()

np.int64(0)

In [28]:
df = df.dropna(subset = ['CustomerID'])

In [30]:
df = df.drop_duplicates()

In [32]:
df['Revenue'] = df['Quantity'] * df['UnitPrice']

In [34]:
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day

In [53]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Year,Month,Day
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30,2010,12,1
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34,2010,12,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00,2010,12,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34,2010,12,1
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34,2010,12,1


In [48]:
df.to_sql('Cleaned_Retail_data', engine, if_exists='append', index=False, schema='online_retails', method= 'multi', chunksize=5000)

392731

Analysis

In [57]:
# Time Series (2011 Revenue by Month)

# Filter data for Year = 2011
# Group by month and calculate revenue totals
# Identify seasonal patterns or month-to-month changes

data_2011 = df[df['Year'] == 2011]
data_2011

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Year,Month,Day
42481,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04,1.95,13313.0,United Kingdom,19.50,2011,1,4
42482,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04,0.42,13313.0,United Kingdom,10.50,2011,1,4
42483,539993,21498,RED RETROSPOT WRAP,25,2011-01-04,0.42,13313.0,United Kingdom,10.50,2011,1,4
42484,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04,2.10,13313.0,United Kingdom,10.50,2011,1,4
42485,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04,1.25,13313.0,United Kingdom,12.50,2011,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09,0.85,12680.0,France,10.20,2011,12,9
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09,2.10,12680.0,France,12.60,2011,12,9
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09,4.15,12680.0,France,16.60,2011,12,9
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09,4.15,12680.0,France,16.60,2011,12,9


In [64]:
monthly_revenue = (data_2011.groupby("Month")['Revenue'].sum())
monthly_revenue

Month
1      568101.310
2      446084.920
3      594081.760
4      468374.331
5      677355.150
6      660046.050
7      598962.901
8      644051.040
9      950686.302
10    1035642.450
11    1156205.610
12     517190.440
Name: Revenue, dtype: float64

In [68]:
monthly_revenue['Pct_change'] = monthly_revenue.pct_change() * 100
monthly_revenue['Pct_change'] 

Month
1                                                           NaN
2                                                    -21.477928
3                                                     33.176831
4                                                    -21.159954
5                                                     44.618333
6                                                     -2.555395
7                                                     -9.254377
8                                                      7.527701
9                                                     47.610398
10                                                     8.936297
11                                                    11.641388
12                                                   -55.268299
PCT_CHANGE    Month
1                                       ...
Name: Revenue, dtype: object

In [72]:
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Greece', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Israel', 'Saudi Arabia', 'Czech Republic',
       'Canada', 'Unspecified', 'Brazil', 'USA', 'European Community',
       'Bahrain', 'Malta', 'RSA'], dtype=object)

In [78]:
# Country Performance (Excluding United Kingdom)

# Rank countries by total revenue
# Identify the top 10 revenue-generating countries
# Compute both revenue and quantity sold

countries = df[df['Country']!='United Kingdom']
countries['Country'].unique()

array(['France', 'Australia', 'Netherlands', 'Germany', 'Norway', 'EIRE',
       'Switzerland', 'Spain', 'Poland', 'Portugal', 'Italy', 'Belgium',
       'Lithuania', 'Japan', 'Iceland', 'Channel Islands', 'Denmark',
       'Cyprus', 'Sweden', 'Finland', 'Austria', 'Greece', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Israel', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Bahrain', 'Malta', 'RSA'], dtype=object)

In [82]:
revenue_by_country = countries.groupby('Country')['Revenue'].sum().sort_values(ascending=False)
revenue_by_country.head(10)

Country
Netherlands    285446.34
EIRE           265262.46
Germany        228678.40
France         208934.31
Australia      138453.81
Spain           61558.56
Switzerland     56443.95
Belgium         41196.34
Sweden          38367.83
Japan           37416.37
Name: Revenue, dtype: float64

In [89]:
revenue_and_qty = countries.groupby('Country').agg(
    total_revenue = ('Revenue', 'sum'),
    total_qty = ('Quantity', 'sum')
).sort_values('total_revenue', ascending=False)
revenue_and_qty.head()

,total_revenue,total_qty
Country,,
Netherlands,285446.34,200937
EIRE,265262.46,140383
Germany,228678.40,119156
France,208934.31,111429
Australia,138453.81,84199


In [ ]:
# Top Customers by Revenue
# Rank all customers by total revenue
# Identify the top 10 highest-spending customers

In [94]:
customers = (df.groupby('CustomerID')['Revenue']).sum().reset_index()
customers


,CustomerID,Revenue
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40
...,...,...
4334,18280.0,180.60
4335,18281.0,80.82
4336,18282.0,178.05
4337,18283.0,2045.53


In [96]:
top_customers = customers.head(10)
top_customers

,CustomerID,Revenue
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40
5,12352.0,2506.04
6,12353.0,89.00
7,12354.0,1079.40
8,12355.0,459.40
9,12356.0,2811.43


In [97]:
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Year,Month,Day
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30,2010,12,1
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34,2010,12,1


In [ ]:
# Global Product Demand

# Compute total quantity sold per country
# Remove the United Kingdom
# Rank countries by demand and highlight high-opportunity markets

In [ ]:
TotalQty_per_country = (countries.groupby('Country')['Quantity']).sum()
TotalQty_per_country.sort_values(ascending=False).reset_index()



,Country,Quantity
0,Netherlands,200937
1,EIRE,140383
2,Germany,119156
3,France,111429
4,Australia,84199
5,Sweden,36078
6,Switzerland,30083
7,Spain,27944
8,Japan,26016
9,Belgium,23237
